# Planck cluster counts

In [1]:
import numpy as np
from classy_sz import Class as Class_sz

## Binned calculation

In [2]:
%%time
class_sz = Class_sz()


class_sz.set({

'output' : 'sz_cluster_counts',
'mass function' :  'T08M500c',
'has_selection_function' : 1,
'experiment' : 0,
'y_m_relation' : 0,
'use_skyaveraged_noise': 0,
'use_planck_binned_proba' : 0,


'm_pivot_ym_[Msun]': 3e14,    

'M_min' : 1e13*0.7, 
'M_max' : 1e16*0.7,

'z_min' : 0.01,
'z_max' : 1.02,

'omega_b': 0.0224178568132,
'omega_cdm':  0.11933148326520002,
'H0': 70.,
'tau_reio': 0.0561,
'ln10^{10}A_s': 2.9799585,
'n_s': 0.96,



'bin_z_min_cluster_counts' : 0.01,
'bin_z_max_cluster_counts' : 1.01,
'bin_dz_cluster_counts' : 0.1,

'bin_dlog10_snr': 0.25, # fiducial 0.25
'log10_snr_min' : 0.7, # fiducial 0.7

#the paramater dlny is crucial, it controls the speed of the calculation
#important to check that lnymin and lnymax are broad enough
'dlny' : 0.01, # fiducial 0.05
'lnymin' : -15, # fiducial -11
'lnymax' : 1., # fiducial 1.
'sigmaM_ym' :0.173, # fiducial 0.173


'dlnM_cluster_count_completeness_grid' : 0.01, # 0.01 fiducial


'cluster_count_completeness_grid_z_cutoff_low' : 0.5,
'cluster_count_completeness_grid_z_cutoff_mid' : 1.5,
'dz_cluster_count_completeness_grid_low_z' : 5e-3,
'dz_cluster_count_completeness_grid_mid_z' : 1e-2,
'dz_cluster_count_completeness_grid_high_z' : 1e-1,





'signal-to-noise_cut-off_for_survey_cluster_completeness' : 6.,


# X ray mass bias (if applicable)
'B' : 1.25,




# tabulation of mass function:
'n_z_dndlnM' : 200,
'n_m_dndlnM' : 200,
'ndim_redshifts' : 200,

'szcc_dof': 0.,
'cosmo_model': 1,

})
class_sz.compute_class_szfast()

/Users/boris/Work/CLASS-SZ/SO-SZ/mcfit/mcfit/mcfit.py:130: UserWarning: use backend='jax' if desired
  warnings.warn("use backend='jax' if desired")


CPU times: user 10.2 s, sys: 2.04 s, total: 12.3 s
Wall time: 2.53 s


In [3]:
dNdzdy_theoretical = class_sz.dndzdy_theoretical()['dndzdy']
z_center = class_sz.dndzdy_theoretical()['z_center']
z_edges = class_sz.dndzdy_theoretical()['z_edges']
log10y_center = class_sz.dndzdy_theoretical()['log10y_center']
log10y_edges = class_sz.dndzdy_theoretical()['log10y_edges']

N_z,N_y = np.shape(dNdzdy_theoretical)
N_clusters_z_theory = []

for iz in range(N_z):
    N_clusters_z_theory.append(np.sum(dNdzdy_theoretical[iz][0:]))
N_clusters_y_theory = []

for iy in range(N_y):
    N_clusters_y_theory.append(np.sum(np.asarray(dNdzdy_theoretical)[:,iy]))
print('Ntot %.10e'%(np.sum(N_clusters_z_theory)))

Ntot 5.1298314215e+02


In [4]:
class_sz.sigma8()

0.787079475530407

## Unbinned calculation

See [Zubeldia & Bolliet (2024)](https://inspirehep.net/literature/2768906).

In [15]:
%%time
class_sz = Class_sz()
class_sz.set({

'output' : 'sz_cluster_counts_fft',
'mass function' :  'T08M500c',
'has_selection_function' : 1,
'experiment' : 0,
'y_m_relation' : 0,
'use_skyaveraged_noise': 0,
'use_planck_binned_proba' : 0,


'm_pivot_ym_[Msun]': 3e14,    

'M_min' : 1e13*0.7, 
'M_max' : 1e16*0.7,

'z_min' : 0.01,
'z_max' : 1.02,

'omega_b': 0.0224178568132,
'omega_cdm':  0.11933148326520002,
'H0': 70.,
'tau_reio': 0.0561,
'ln10^{10}A_s': 2.9799585,
'n_s': 0.96,



'sigmaM_ym' :0.173, # fiducial 0.173


'szcc_dof': 0.0,
'szcc_qtrunc': -1.0,
'szcounts_fft_nz': 150,
'szcounts_fft_z_max': 1.01,
'szcounts_fft_z_min': 0.01,
'szcounts_qmax_fft_padded': 500.0,
'N_samp_fftw': 1024,


'signal-to-noise_cut-off_for_survey_cluster_completeness' : 6.,


# X ray based hydrostatic mass mass bias (if applicable)
'B' : 1.25,
'cosmo_model': 1,

})
class_sz.compute_class_szfast()


CPU times: user 59 s, sys: 1.56 s, total: 1min
Wall time: 6.66 s


In [16]:
zmin = class_sz.pars['szcounts_fft_z_min']
zmax = class_sz.pars['szcounts_fft_z_max']
nz = class_sz.pars['szcounts_fft_nz']
z_arr = np.linspace(zmin,zmax,nz)

q_threshold = 6.
q_max = 100.
nq = 5000
q_arr = np.geomspace(q_threshold, q_max,nq)

get_dndzdq = np.vectorize(class_sz.get_szcounts_dndzdq_at_z_q)

In [17]:
Nz = []
for zp in z_arr:
    Nz.append(np.trapz(get_dndzdq(zp,q_arr)*q_arr,x=np.log(q_arr)))
Nz = np.asarray(Nz)

Nq = []
Nq = np.asarray(Nq)

In [18]:
Ntot = np.trapz(Nz,x=z_arr)
Ntot

523.5737819932679

In [19]:
class_sz.sigma8()

0.787079475530407